### 回写指令

In [ ]:
#coding:gbk

import cx_Oracle, os, csv
import pandas as pd
import numpy as np
from pandas import DataFrame
conn = cx_Oracle.connect(' ',' ',' ') # sql info

def _exec_sql(sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    conn.commit()
    
def Update_SQL(new_DETLASHARE,accountid,SoF,dte,tkr):
    updatesql = " update instructs_algo set DETLASHARE={} where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(new_DETLASHARE,accountid,SoF,dte,tkr)
    _exec_sql(updatesql)
    sql = " select dte,accountid,tkr,detlashare from instructs_algo where accountid like '%{}%{}' and dte={} and tkr='{}' and traderid=102".format(accountid,SoF,dte,tkr)
    df_instr = pd.read_sql(sql,conn)
    print(df_instr.values)
    
def Modify_instr(dte, accountid, SoF):
    # 实际
    sql = "select tkr,sum(qty),side from transactions_algo where dte={} and accountid like '%{}%{}' group by tkr,side".format(dte,accountid,SoF)
    df_trans = pd.read_sql(sql,conn)
    for i in range(df_trans.shape[0]):
        if df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'S':
            new_DETLASHARE = -df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
        elif df_trans.values[i][np.where(df_trans.columns == 'SIDE')][0] == 'B':
            new_DETLASHARE = df_trans.values[i][np.where(df_trans.columns == 'SUM(QTY)')][0]
        tkr = df_trans.values[i][np.where(df_trans.columns == 'TKR')][0]
        print(df_trans.values[i])
        print(new_DETLASHARE)
        Update_SQL(new_DETLASHARE,accountid,SoF,dte,tkr)
    print('done!')
    
if __name__ == '__main__':

    dte = str(20201218)
    accountid = 'CY53H'
    SoF = 'S'
    
    Modify_instr(dte, accountid, SoF)